# Input Data

In [1]:
from __future__ import print_function

In [2]:
from salib import extend, import_notebooks
import_notebooks()
from Tables import Table
from Nodes import Node
from Members import Member
from LoadSets import LoadSet, LoadCombination
from NodeLoads import makeNodeLoad
from MemberLoads import makeMemberLoad
from collections import OrderedDict, defaultdict
import numpy as np

In [3]:
from Frame2D_Base import Frame2D

In [4]:
@extend
class Frame2D:
    
    COLUMNS_xxx = [] # list of column names for table 'xxx'
        
    def get_table(self,tablename,extrasok=False,optional=False):
        columns = getattr(self,'COLUMNS_'+tablename)
        t = Table(tablename,ds_name=self.dsname,columns=columns,optional=optional)
        t.read(optional=optional)
        reqdl= columns
        reqd = set(reqdl)
        prov = set(t.columns)
        if reqd-prov:
            raise Exception('Columns missing {} for table "{}". Required columns are: {}'\
                            .format(list(reqd-prov),tablename,reqdl))
        if not extrasok:
            if prov-reqd:
                raise Exception('Extra columns {} for table "{}". Required columns are: {}'\
                               .format(list(prov-reqd),tablename,reqdl))
        return t

## Test Frame
![test frame](img/frame-6b.svg)
## Nodes

In [5]:
%%Table nodes
NODEID,X,Y,Z
A,0.,0.,5000.
B,0,4000,5000
C,8000,4000,5000
D,8000,0,5000

In [6]:
@extend
class Frame2D:
    
    COLUMNS_nodes = ('NODEID','X','Y')
        
    def input_nodes(self):
        node_table = self.get_table('nodes')
        for ix,r in node_table.data.iterrows():
            if r.NODEID in self.nodes:
                raise Exception('Multiply defined node: {}'.format(r.NODEID))
            n = Node(r.NODEID,r.X,r.Y)
            self.nodes[n.id] = n
        self.rawdata.nodes = node_table
            
    def get_node(self,id):
        try:
            return self.nodes[id]
        except KeyError:
            raise Exception('Node not defined: {}'.format(id))


In [7]:
##test:
f = Frame2D()

In [8]:
##test:
f.input_nodes()

In [9]:
##test:
f.nodes

OrderedDict([('A', Node("A",0.0,0.0)),
             ('B', Node("B",0.0,4000.0)),
             ('C', Node("C",8000.0,4000.0)),
             ('D', Node("D",8000.0,0.0))])

In [10]:
##test:
f.get_node('C')

Node("C",8000.0,4000.0)

## Supports

In [11]:
%%Table supports
NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY

In [12]:
def isnan(x):
    if x is None:
        return True
    try:
        return np.isnan(x)
    except TypeError:
        return False

In [13]:
@extend
class Frame2D:
    
    COLUMNS_supports = ('NODEID','C0','C1','C2')
    
    def input_supports(self):
        table = self.get_table('supports')
        for ix,row in table.data.iterrows():
            node = self.get_node(row.NODEID)
            for c in [row.C0,row.C1,row.C2]:
                if not isnan(c):
                    node.add_constraint(c)
        self.rawdata.supports = table

In [14]:
##test:
f.input_supports()

In [15]:
##test:
vars(f.get_node('D'))

{'constraints': {'FX', 'FY'},
 'dofnums': [None, None, None],
 'id': 'D',
 'x': 8000.0,
 'y': 0.0}

## Members

In [16]:
%%Table members
MEMBERID,NODEJ,NODEK
AB,A,B
BC,B,C
CD,C,D

In [17]:
@extend
class Frame2D:
    
    COLUMNS_members = ('MEMBERID','NODEJ','NODEK')
    
    def input_members(self):
        table = self.get_table('members')
        for ix,m in table.data.iterrows():
            if m.MEMBERID in self.members:
                raise Exception('Multiply defined member: {}'.format(m.MEMBERID))
            memb = Member(m.MEMBERID,self.get_node(m.NODEJ),self.get_node(m.NODEK))
            self.members[memb.id] = memb
        self.rawdata.members = table
            
    def get_member(self,id):
        try:
            return self.members[id]
        except KeyError:
            raise Exception('Member not defined: {}'.format(id))

In [18]:
##test:
f.input_members()
f.members

OrderedDict([('AB', Member("AB","Node("A",0.0,0.0)","Node("B",0.0,4000.0)")),
             ('BC',
              Member("BC","Node("B",0.0,4000.0)","Node("C",8000.0,4000.0)")),
             ('CD',
              Member("CD","Node("C",8000.0,4000.0)","Node("D",8000.0,0.0)"))])

In [19]:
##test:
m = f.get_member('BC')
m.id, m.L, m.dcx, m.dcy

('BC', 8000.0, 1.0, 0.0)

## Releases

In [20]:
%%Table releases
MEMBERID,RELEASE
AB,MZK
CD,MZJ

In [21]:
@extend
class Frame2D:
    
    COLUMNS_releases = ('MEMBERID','RELEASE')
    
    def input_releases(self):
        table = self.get_table('releases',optional=True)
        for ix,r in table.data.iterrows():
            memb = self.get_member(r.MEMBERID)
            memb.add_release(r.RELEASE)
        self.rawdata.releases = table

In [22]:
##test:
f.input_releases()

In [23]:
##test:
vars(f.get_member('AB'))

{'A': None,
 'Ix': None,
 'KG': None,
 'KL': None,
 'L': 4000.0,
 'Tm': None,
 'dcx': 0.0,
 'dcy': 1.0,
 'id': 'AB',
 'nodej': Node("A",0.0,0.0),
 'nodek': Node("B",0.0,4000.0),
 'releases': {'MZK'}}

## Properties

If the SST module is loadable, member properties may be specified by giving steel shape designations
(such as 'W310x97') in the member properties data.  If the module is not available, you may still give $A$ and
$I_x$ directly (it only tries to lookup the properties if these two are not provided).

In [24]:
try:
    from sst import SST
    __SST = SST()
    get_section = __SST.section
except ImportError:
    def get_section(dsg,fields):
        raise ValueError('Cannot lookup property SIZE because SST is not available.  SIZE = {}'.format(dsg))
        ##return [1.] * len(fields.split(',')) # in case you want to do it that way

In [25]:
%%Table properties
MEMBERID,SIZE,IX,A
BC,W460x106,,
AB,W310x97,,
CD,,

In [26]:
@extend
class Frame2D:
    
    COLUMNS_properties = ('MEMBERID','SIZE','IX','A')
    
    def input_properties(self):
        table = self.get_table('properties')
        table = self.fill_properties(table)
        for ix,row in table.data.iterrows():
            memb = self.get_member(row.MEMBERID)
            memb.size = row.SIZE
            memb.Ix = row.IX
            memb.A = row.A
        self.rawdata.properties = table
        
    def fill_properties(self,table):
        data = table.data
        prev = None
        for ix,row in data.iterrows():
            nf = 0
            if type(row.SIZE) in [type(''),type(u'')]:
                if isnan(row.IX) or isnan(row.A):
                    Ix,A = get_section(row.SIZE,'Ix,A')
                    if isnan(row.IX):
                        nf += 1
                        data.loc[ix,'IX'] = Ix
                    if isnan(row.A):
                        nf += 1
                        data.loc[ix,'A'] = A
            elif isnan(row.SIZE):
                data.loc[ix,'SIZE'] = '' if nf == 0 else prev
            prev = data.loc[ix,'SIZE']
        table.data = data.fillna(method='ffill')
        return table

In [27]:
##test:
f.input_properties()

In [28]:
##test:
vars(f.get_member('CD'))

{'A': 12300.0,
 'Ix': 222000000.0,
 'KG': None,
 'KL': None,
 'L': 4000.0,
 'Tm': None,
 'dcx': 0.0,
 'dcy': -1.0,
 'id': 'CD',
 'nodej': Node("C",8000.0,4000.0),
 'nodek': Node("D",8000.0,0.0),
 'releases': {'MZJ'},
 'size': ''}

## Node Loads

In [29]:
%%Table node_loads
LOAD,NODEID,DIRN,F
Wind,B,FX,-200000.

In [30]:
@extend
class Frame2D:
    
    COLUMNS_node_loads = ('LOAD','NODEID','DIRN','F')
    
    def input_node_loads(self):
        table = self.get_table('node_loads')
        dirns = ['FX','FY','FZ']
        for ix,row in table.data.iterrows():
            n = self.get_node(row.NODEID)
            if row.DIRN not in dirns:
                raise ValueError("Invalid node load direction: {} for load {}, node {}; must be one of '{}'"
                                .format(row.DIRN, row.LOAD, row.NODEID, ', '.join(dirns)))
            if row.DIRN in n.constraints:
                raise ValueError("Constrained node {} {} must not have load applied."
                                .format(row.NODEID,row.DIRN))
            l = makeNodeLoad({row.DIRN:row.F})
            self.nodeloads.append(row.LOAD,n,l)
        self.rawdata.node_loads = table

In [31]:
##test:
f.input_node_loads()

In [32]:
##test:
for o,l,fact in f.nodeloads.iterloads('Wind'):
    print(o,l,fact,l*fact)

Node("B",0.0,4000.0) NodeLoad(-200000.0,0.0,0.0) 1.0 NodeLoad(-200000.0,0.0,0.0)


## Support Displacements

In [33]:
%%Table support_displacements
LOAD,NODEID,DIRN,DELTA
Other,A,DY,-10

In [34]:
@extend
class Frame2D:
    
    COLUMNS_support_displacements = ('LOAD','NODEID','DIRN','DELTA')
    
    def input_support_displacements(self):
        table = self.get_table('support_displacements',optional=True)
        forns = {'DX':'FX','DY':'FY','TZ':'MZ'}
        for ix,row in table.data.iterrows():
            n = self.get_node(row.NODEID)
            if row.DIRN not in forns:
                raise ValueError("Invalid support displacements direction: {} for load {}, node {}; must be one of '{}'"
                                .format(row.DIRN, row.LOAD, row.NODEID, ', '.join(forns.keys())))
            fd = forns[row.DIRN]
            if fd not in n.constraints:
                raise ValueError("Support displacement, load: '{}'  node: '{}'  dirn: '{}' must be for a constrained node."
                                .format(row.LOAD,row.NODEID,row.DIRN))
            l = makeNodeLoad({fd:row.DELTA})
            self.nodedeltas.append(row.LOAD,n,l)
        self.rawdata.support_displacements = table

In [35]:
##test:
f.input_support_displacements()

In [36]:
##test:
list(f.nodedeltas)[0]

('other', Node("A",0.0,0.0), NodeLoad(0.0,-10.0,0.0))

## Member Loads

In [37]:
%%Table member_loads
LOAD,MEMBERID,TYPE,W1,W2,A,B,C
Live,BC,UDL,-50,,,,
Live,BC,PL,-200000,,5000

In [38]:
@extend
class Frame2D:
    
    COLUMNS_member_loads = ('LOAD','MEMBERID','TYPE','W1','W2','A','B','C')
    
    def input_member_loads(self):
        table = self.get_table('member_loads')
        for ix,row in table.data.iterrows():
            m = self.get_member(row.MEMBERID)
            l = makeMemberLoad(m.L,row)
            self.memberloads.append(row.LOAD,m,l)
        self.rawdata.member_loads = table

In [39]:
##test:
f.input_member_loads()

In [40]:
##test:
for o,l,fact in f.memberloads.iterloads('Live'):
    print(o.id,l,fact,l.fefs()*fact)

BC UDL(L=8000.0,w=-50) 1.0 EF(0.0,200000.0,266666666.667,0.0,200000.0,-266666666.667)
BC PL(L=8000.0,P=-200000,a=5000.0) 1.0 EF(0.0,63281.25,140625000.0,0.0,136718.75,-234375000.0)


## Load Combinations

In [41]:
%%Table load_combinations
CASE,LOAD,FACTOR
One,Live,1.5
One,Wind,1.75

In [42]:
@extend
class Frame2D:
    
    COLUMNS_load_combinations = ('CASE','LOAD','FACTOR')
    
    def input_load_combinations(self):
        table = self.get_table('load_combinations',optional=True)
        if len(table) > 0:
            for ix,row in table.data.iterrows():
                self.loadcombinations.append(row.CASE,row.LOAD,row.FACTOR)
        if 'all' not in self.loadcombinations:
            all = self.nodeloads.names.union(self.memberloads.names)
            all = self.nodedeltas.names.union(all)
            for l in all:
                self.loadcombinations.append('all',l,1.0)
        self.rawdata.load_combinations = table

In [43]:
##test:
f.input_load_combinations()

In [44]:
##test:
for o,l,fact in f.loadcombinations.iterloads('One',f.nodeloads):
    print(o.id,l,fact)
for o,l,fact in f.loadcombinations.iterloads('One',f.memberloads):
    print(o.id,l,fact,l.fefs()*fact)

B NodeLoad(-200000.0,0.0,0.0) 1.75
BC UDL(L=8000.0,w=-50) 1.5 EF(0.0,300000.0,400000000.0,0.0,300000.0,-400000000.0)
BC PL(L=8000.0,P=-200000,a=5000.0) 1.5 EF(0.0,94921.875,210937500.0,0.0,205078.125,-351562500.0)


### Load Iterators

In [45]:
@extend
class Frame2D:

    def iter_nodeloads(self,casename):
        for o,l,f in self.loadcombinations.iterloads(casename,self.nodeloads):
            yield o,l,f
    
    def iter_nodedeltas(self,casename):
        for o,l,f in self.loadcombinations.iterloads(casename,self.nodedeltas):
            yield o,l,f
    
    def iter_memberloads(self,casename):
        for o,l,f in self.loadcombinations.iterloads(casename,self.memberloads):
            yield o,l,f

In [46]:
##test:
for o,l,fact in f.iter_nodeloads('One'):
    print(o.id,l,fact)
for o,l,fact in f.iter_memberloads('One'):
    print(o.id,l,fact)

B NodeLoad(-200000.0,0.0,0.0) 1.75
BC UDL(L=8000.0,w=-50) 1.5
BC PL(L=8000.0,P=-200000,a=5000.0) 1.5


## Number the DOFs

In [47]:
@extend
class Frame2D:
    
    def number_dofs(self):
        self.ndof = (3*len(self.nodes))
        self.ncons = sum([len(node.constraints) for node in self.nodes.values()])
        self.nfree = self.ndof - self.ncons
        ifree = 0
        icons = self.nfree
        self.dofdesc = [None] * self.ndof
        for node in self.nodes.values():
            for dirn,ix in node.DIRECTIONS.items():
                if dirn in node.constraints:
                    n = icons
                    icons += 1
                else:
                    n = ifree
                    ifree += 1
                node.dofnums[ix] = n
                self.dofdesc[n] = (node,dirn)

In [48]:
##test:
f.number_dofs()
f.ndof, f.ncons, f.nfree

(12, 5, 7)

In [49]:
##test:
f.dofdesc

[(Node("B",0.0,4000.0), 'FX'),
 (Node("B",0.0,4000.0), 'FY'),
 (Node("B",0.0,4000.0), 'MZ'),
 (Node("C",8000.0,4000.0), 'FX'),
 (Node("C",8000.0,4000.0), 'FY'),
 (Node("C",8000.0,4000.0), 'MZ'),
 (Node("D",8000.0,0.0), 'MZ'),
 (Node("A",0.0,0.0), 'FX'),
 (Node("A",0.0,0.0), 'FY'),
 (Node("A",0.0,0.0), 'MZ'),
 (Node("D",8000.0,0.0), 'FX'),
 (Node("D",8000.0,0.0), 'FY')]

In [50]:
##test:
f.get_node('D').dofnums

[10, 11, 6]

## Input Everything

In [51]:
@extend
class Frame2D:
    
    def input_all(self):
        self.input_nodes()
        self.input_supports()
        self.input_members()
        self.input_releases()
        self.input_properties()
        self.input_node_loads()
        self.input_support_displacements()
        self.input_member_loads()
        self.input_load_combinations()
        self.input_finish()
        
    def input_finish(self):
        self.number_dofs()

In [52]:
##test:
f.reset()
f.input_all()

## Accumulated Cell Data

In [53]:
##test:
Table.CELLDATA

{u'load_combinations': u'CASE,LOAD,FACTOR\nOne,Live,1.5\nOne,Wind,1.75',
 u'member_loads': u'LOAD,MEMBERID,TYPE,W1,W2,A,B,C\nLive,BC,UDL,-50,,,,\nLive,BC,PL,-200000,,5000',
 u'members': u'MEMBERID,NODEJ,NODEK\nAB,A,B\nBC,B,C\nCD,C,D',
 u'node_loads': u'LOAD,NODEID,DIRN,F\nWind,B,FX,-200000.',
 u'nodes': u'NODEID,X,Y,Z\nA,0.,0.,5000.\nB,0,4000,5000\nC,8000,4000,5000\nD,8000,0,5000',
 u'properties': u'MEMBERID,SIZE,IX,A\nBC,W460x106,,\nAB,W310x97,,\nCD,,',
 u'releases': u'MEMBERID,RELEASE\nAB,MZK\nCD,MZJ',
 u'support_displacements': u'LOAD,NODEID,DIRN,DELTA\nOther,A,DY,-10',
 u'supports': u'NODEID,C0,C1,C2\nA,FX,FY,MZ\nD,FX,FY'}

## Input From Files

In [54]:
##test:
f.reset()
Table.set_source('frame-1')
f.input_all()

In [55]:
##test:
vars(f.rawdata)

{'load_combinations': <Tables.Table at 0x7fd4a76a5d90>,
 'member_loads': <Tables.Table at 0x7fd4a76a5590>,
 'members': <Tables.Table at 0x7fd4a7697ed0>,
 'node_loads': <Tables.Table at 0x7fd4a73fee90>,
 'nodes': <Tables.Table at 0x7fd4a74445d0>,
 'properties': <Tables.Table at 0x7fd4a76a5090>,
 'releases': <Tables.Table at 0x7fd4a76a5c10>,
 'support_displacements': <Tables.Table at 0x7fd4a7444150>,
 'supports': <Tables.Table at 0x7fd4a7697e90>}

In [56]:
##test:
f.rawdata.nodes.data

,NODEID,X,Y
0,A,0,0
1,B,0,4000
2,C,8000,4000
3,D,8000,0


In [57]:
##test:
f.members

OrderedDict([('AB', Member("AB","Node("A",0.0,0.0)","Node("B",0.0,4000.0)")),
             ('BC',
              Member("BC","Node("B",0.0,4000.0)","Node("C",8000.0,4000.0)")),
             ('CD',
              Member("CD","Node("C",8000.0,4000.0)","Node("D",8000.0,0.0)"))])

In [58]:
##test:
Table.CELLDATA

{}